# Representations of Color
Converting input Hex codes to RGB and HSL.

In [1]:
!pip install pycolorkit

In [2]:
import pandas as pd
from pycolorkit.pycolorkit import ColorConverter, ColorGenerator

In [3]:
# Valid entries
df = pd.DataFrame(['#FF0000', '#00ff00', '#fff'], columns=['hex'])
df.head()

,hex
0,#FF0000
1,#00ff00
2,#fff


## Hex to RGB
Convert every set of two hexidecimals to decimals.

In [4]:
df['hex'] = df['hex'].apply(ColorConverter.format_hex)
df

,hex
0,FF0000
1,00ff00
2,ffffff


In [5]:
df['rgb'] = df['hex'].apply(lambda h: list(ColorConverter.hex_to_rgb(h)))
df

,hex,rgb
0,FF0000,"[255, 0, 0]"
1,00ff00,"[0, 255, 0]"
2,ffffff,"[255, 255, 255]"


## RGB to HSL
This human-friendly conversion allows us to represent color on a color wheel by determining where the color point exists in relation to pure red, green, or blue. We can do this in 4 steps:

**Step 1**. Calculate the max, min, and chroma.  

$
M = max(R, G, B) \\
m = min(R, G, B) \\
C = range(R, G, B) = M - m$

**Step 2**. Calculate the hue measured in degrees `[0°, 360°)`.  

$H' = 
\begin{cases}
\text{undefined}, & \text{if } C = 0 \\
\frac{G-B}{C} \bmod 6, & \text{if } M = R \\
\frac{B-R}{C} + 2, & \text{if } M = G \\
\frac{R-G}{C} + 4, & \text{if } M = B
\end{cases}$

Then, $H = 60° × H'$

**Step 3**. Calculate Lightness measuring color intensity using set `[0%, 100%)`.

$L = mid(R, G, B) = \frac{1}{2}(M + m)$
    

**Step 4**. Calculate Saturation determined by the amount of grayscale or pure color from `[0%, 100%)`.

$S_L =
\begin{cases}
0, & \text{if } L = 1 \text{ or } L = 0 \\
\frac{C}{1-|2L-1|}, & \text{otherwise}
\end{cases}$
  

See [wiki page](https://en.wikipedia.org/wiki/HSL_and_HSV).

In [6]:
df['hsl'] = df['rgb'].apply(lambda rgb: list(ColorConverter.rgb_to_hsl(rgb)))
df

,hex,rgb,hsl
0,FF0000,"[255, 0, 0]","[0, 100, 50]"
1,00ff00,"[0, 255, 0]","[120, 100, 50]"
2,ffffff,"[255, 255, 255]","[0, 0, 100]"


# Lightness Adjustments
Center the base lightness value and generate a range of shades and tints in the same hue. Tints add light to the lightness (`l`). Shades removes light from the lightness (`l`). Functions assume that l is a decimal value. The base color value is included in the final output.

In [7]:
df['sequential_hsl'] = df['hsl'].apply(ColorGenerator.sequence)
df[['hsl','sequential_hsl']]

,hsl,sequential_hsl
0,"[0, 100, 50]","[(0, 100, 10), (0, 100, 18), (0, 100, 26), (0,..."
1,"[120, 100, 50]","[(120, 100, 10), (120, 100, 18), (120, 100, 26..."
2,"[0, 0, 100]",None


# Complimentary Colors
Colors compliment each other when they are directly opposite one another on the color wheel. Add $180\degree$ to the base color hue value.

In [8]:
def compliment(hsl):
    h = hsl[0]
    s = hsl[1]
    l = hsl[2]

    if (h + 180) > 360:
        return (h-180, s, l)
    else:
        return (h+180, s, l)

In [9]:
df['compliment_hsl'] = df['hsl'].apply(lambda hsl: list(ColorGenerator.compliment(hsl)))
df[['hsl','compliment_hsl']]

,hsl,compliment_hsl
0,"[0, 100, 50]","[180, 100, 50]"
1,"[120, 100, 50]","[300, 100, 50]"
2,"[0, 0, 100]","[180, 0, 100]"


You can also create a sequence of colors between the base colors and it's complimentary colors since they have th same lightness values.

In [ ]:
# sequence lightness for complimentary colors
df['diverging_hsl'] = df['compliment_hsl'].apply(ColorGenerator.sequence)

# reverse list order
df['diverging_hsl'] = df['diverging_hsl'].apply(lambda x: try: x[::-1] except: x)

# combine with base color sequence
df['diverging_hsl'] = df['diverging_hsl'] + df['sequential_hsl']

# preview colors
print([i for i in df[['diverging_hsl']].iloc[0]])

TypeError: 'NoneType' object is not subscriptable

# HSL to Hex
Returning generated colors to machine compatible color codes.

### HSL to RGB
Step 1. Find chroma.

$C = (1 - |2L - 1|) * S_L$
  

Step 2. Find a point (R,G,B) with the same hue and chroma.

$H' = \frac{H}{60\degree}$
    
$X = C s (1 |H' \mod 2 - 1|) \\$  
 
$(R_1, G_1, B_1) = 
\begin{cases}
(C, X, 0) & \text{if } 0<=H'<1 \\
(X, C, 0) & \text{if } 1<=H'<2 \\
(0, C, X) & \text{if } 2<=H'<3 \\
(0, X, C) & \text{if } 3<=H'<4 \\
(X, 0, C) & \text{if } 4<=H'<5 \\
(C, 0, X) & \text{if } 5<=H'<6 \\
\end{cases}$

Step 3. Add amount to match the lightness.

$m = L - \frac{C}{2}$  
 
$(R, G, B) = (R_1 + m, G_1 + m, B_1 + m)$

In [ ]:
df['sequential_rgb'] = df['sequential_hsl'].apply(lambda hsl: [ColorConverter.hsl_to_rgb(i) for i in hsl])
df[['sequential_rgb']]

In [ ]:
df['diverging_rgb'] = df['diverging_hsl'].apply(lambda hsl: [ColorConverter.hsl_to_rgb(i) for i in hsl])
df[['diverging_rgb']]

### RGB to Hex

In [ ]:
df['sequential_hex'] = df['sequential_rgb'].apply(lambda rgb: [ColorConverter.rgb_to_hex(i) for i in rgb])
df[['sequential_hex']]

In [ ]:
df['diverging_hex'] = df['diverging_rgb'].apply(lambda rgb: [ColorConverter.rgb_to_hex(i) for i in rgb])
df[['diverging_hex']]

Preview all conversions.

In [ ]:
df.to_json('./color.json')
df